In [17]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as expected_conditions
import time
from pynput.keyboard import Key, Controller
import genanki
import pandas as pd
import pytesseract
from PIL import Image
from googletrans import Translator
import re
import base64
import os
from urllib.parse import quote as urlquote
from flask import Flask, send_from_directory
import dash
from dash import dcc
from dash import html, callback_context
from dash.dependencies import Input, Output

In [18]:
#login_id = #enter you login id here
#login_pass =  #enter your password
PHOTO_DIRECTORY = os.getcwd() + "\\Photos\\" #enter your photo directory 
FLASHCARD_DIRECTORY = os.getcwd() + "\\Flashcards\\" #enter your flashcard directory 

if not os.path.exists(PHOTO_DIRECTORY):
    os.makedirs(PHOTO_DIRECTORY)

if not os.path.exists(FLASHCARD_DIRECTORY):
    os.makedirs(FLASHCARD_DIRECTORY)


In [19]:
#function to upload the apkg to ankipro
def upload2ankipro(login_id, login_pass, output_filename):
    browser = webdriver.Chrome()#choose any other webdriver depending on which  browser you have, here chrome is used.
    browser.get(("https://ankipro.net/sign-in"))#link of the site

    xpath_username = '//*[@id="__next"]/div[1]/div/div/div[2]/div/div[2]/div[1]/div[2]/input'
    username = browser.find_element(By.XPATH, xpath_username)#login_id_value is the id of the text field in which id is to be entered.
    username.send_keys(login_id)

    xpath_login = '//*[@id="__next"]/div[1]/div/div/div[2]/div/div[2]/div[2]/div[2]/input'
    username = browser.find_element(By.XPATH, xpath_login)#login_id_value is the id of the text field in which id is to be entered.
    username.send_keys(login_pass)

    xpath_button = '//*[@id="__next"]/div[1]/div/div/div[2]/div/div[4]/div/div/div/div/div' 
    signInButton = browser.find_element(By.XPATH, xpath_button)
    signInButton.click()
    time.sleep(3)
    xpath_newdeckbutton = '//*[@id="__next"]/div[1]/div/div/div/div[1]/div[2]/div/div/div/div/div'
    newdeckbutton = browser.find_element(By.XPATH, xpath_newdeckbutton)
    newdeckbutton.click()
    time.sleep(3)
    xpath_uploaddeckbutton = '/html/body/div[7]/div/div[2]/div/div/div[2]/div[2]/div[2]/div/div[2]/div[1]/div[1]/button[2]/div/div'
    uploaddeckbutton = browser.find_element(By.XPATH, xpath_uploaddeckbutton)
    uploaddeckbutton.click()
    xpath_uploadfile =  '/html/body/div[7]/div/div[2]/div/div/div[2]/div[2]/div[2]/div/div[2]/div[3]/div/div'
    browser.find_element(By.XPATH, xpath_uploadfile).click()
    time.sleep(3)

    keyboard = Controller()
    path = FLASHCARD_DIRECTORY + output_filename
    keyboard.type(path)
    time.sleep(3)
    keyboard.press(Key.enter)
    time.sleep(3)

In [20]:
#Function to translate to english
def translate(words):
    try:
        translator = Translator()
        translate_list = []
        trans = translator.translate(words, dest = 'en') 
        for translation in trans:
            translate_list.append(translation.text)
        return translate_list
    except:
        return None

In [21]:
#function to create the apkg file
def photo2apkg(filename):
    # Open the image file
    image = Image.open(PHOTO_DIRECTORY + '\\' + filename)
    pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'
    # Perform OCR using PyTesseract
    text = pytesseract.image_to_string(image, lang='deu')
    pattern = r'\d+|/|\.b|\.a|\.c|\.|Start|der|die|das'
    text = re.sub(pattern, '', text)
    # Text uprava
    text_list = text.splitlines()
    #zbavit se prazdnych stringu
    text_ohneempty = [i for i in text_list if len(i)>=2]
    #zbavit se stringu s cisly

    text_final = pd.DataFrame(text_ohneempty)
    text_final.columns = ['German']

    # Print the extracted text
    # print(text_final)

    text_final['English'] = translate(text_final['German'].tolist())
    #print(text_final)
    #display(text_final)
    items = []

    for index, row in text_final.iterrows():
        german = row['German']
        english = row['English']
        items.append((german, english))

    my_model = genanki.Model(
    1607392319,
    'Simple Model',
    fields=[
        {'name': 'german'},
        {'name': 'english'},
    ],
    templates=[
        {
            'name': 'Forward Card',
            'qfmt': '{{german}}',
            'afmt': '{{FrontSide}}<hr id="answer">{{english}}',
        },
        {
            'name': 'Reverse Card',
            'qfmt': '{{english}}',
            'afmt': '{{FrontSide}}<hr>{{german}}'
        }
    ])

    my_deck = genanki.Deck(2059400110, filename)

    for i in items:
        my_deck.add_note(genanki.Note(
            model=my_model,
            fields=i))
    filename_output = re.sub(r'\..*$', '', filename) + '.apkg'
    genanki.Package(my_deck).write_to_file(FLASHCARD_DIRECTORY + filename_output)
    return filename_output


In [22]:
# Normally, Dash creates its own Flask server internally. By creating our own,
# we can create a route for downloading files directly:
server = Flask(__name__)
app = dash.Dash(server=server)


@server.route("/download/<path:path>")
def download(path):
    """Serve a file from the upload directory."""
    return send_from_directory(PHOTO_DIRECTORY, path, as_attachment=True)

ALLOWED_TYPES = ( "email", "password", )

app.layout = html.Div(
    [
        html.H1("Create your flashcards"),
        html.H2("Insert your AnkiPro email and password"),
        dcc.Input(id="email", type="email", placeholder="", style={'marginRight':'10px'}),
        dcc.Input(id="password", type="password", placeholder="", style={'marginRight':'10px'}),
        html.Button("Enter", id="enter-button", n_clicks=0, style={"margin-top": "10px"}),  # Enter button
        html.Div(id="email_password"),
        html.H2("Upload photo of the vocabulary"),
        dcc.Upload(
            id="upload-data",
            children=html.Div(
                ["Drag and drop or click to select a file to upload."]
                ),
            style={
                "width": "100%",
                "height": "60px",
                "lineHeight": "60px",
                "borderWidth": "1px",
                "borderStyle": "dashed",
                "borderRadius": "5px",
                "textAlign": "center",
                "margin": "10px",
            },
            multiple=True,
        ),
        html.H2("File List"),
        html.Div(id='output-state'),
        dcc.Loading(
            id="loading-output",
            type="default",
            children=[html.Ul(id="file-list")]
         ),
    ],
    style={"max-width": "500px"}
)


def save_file(name, content):
    """Decode and store a file uploaded with Plotly Dash."""
    data = content.encode("utf8").split(b";base64,")[1]
    with open(os.path.join(PHOTO_DIRECTORY, name), "wb") as fp:
        fp.write(base64.decodebytes(data))

def uploaded_files():
    """List the files in the upload directory."""
    files = []
    for filename in os.listdir(PHOTO_DIRECTORY):
        path = os.path.join(PHOTO_DIRECTORY, filename)
        if os.path.isfile(path):
            files.append(filename)
    return files

def file_download_link(filename):
    """Create a Plotly Dash 'A' element that downloads a file from the app."""
    location = "/download/{}".format(urlquote(filename))
    return html.A(filename, href=location)
login_id = None
login_pass = None

@app.callback(
    Output("email_password", "children"),
    [Input("email", "value"), Input("password", "value")],[Input("enter-button", "n_clicks")]
)
def update_output(given_email, given_password, enter):
    # Save email and password #
    global login_id, login_pass
    triggered_input = callback_context.triggered[0]["prop_id"].split(".")[0]
    if given_email is not None and given_password is not None and triggered_input == "enter-button":
        login_id = given_email
        login_pass = given_password
        return html.P('Thank you!')
    
@app.callback(
    Output("file-list", "children"),
    [Input("upload-data", "filename"), Input("upload-data", "contents")]
)
def update_output(uploaded_filenames, uploaded_file_contents):
    """Save uploaded files and regenerate the file list."""
    if uploaded_filenames is not None and uploaded_file_contents is not None:
        for name, data in zip(uploaded_filenames, uploaded_file_contents):
            save_file(name, data)
            output_filename = photo2apkg(name)
            upload2ankipro(login_id, login_pass, output_filename)

    files = uploaded_files()
    if len(files) == 0:
        return [html.Li("No files yet!")]
    else:
        #for filename in files:
        return [html.Li(file_download_link(filename)) for filename in files]



if __name__ == "__main__":
    app.run_server(debug=True, port=8888)        